**CONNECT_FOUR GAME**

First assignment for the Artificial Intelligence class, 2º Semester of 2º year. 

Game logic and interface implemented, as some search algorithms - A*, Monte Carlo.

Work developed by Lucas Oliveira, Marta Longo and Sara Táboas

**Libraries**

In [17]:
import board
import pygame
import numpy as np
import sys #check if necessary

**Game Logic** 

Description

In [18]:
class Board:
    def __init__(self):
        self.cols = 7
        self.rows = 6
        self.board = []
        self.nullSymbol = '.'
        self.p1Symbol = 'X'
        self.p2Symbol = 'O'
        self.rowTops = [5,5,5,5,5,5,5] #as peças iniciais vão para a posicao (5, col)
        self.populateBoard()
        self.end = False
        
    def __str__(self):
        line = ""
        result = ""
        for i in range(0,6):
            for ii in range(self.cols):
                line += self.board[i][ii]
            result += line + "\n"
            line = ""
        return result

    def populateBoard(self):
        aux = []
        for i in range(self.cols):
            aux.append(self.nullSymbol)
        for i in range(self.rows):
            self.board.append(aux.copy())

    def move(self, collumn, player):
        if player == 1:
            self.addToCollumn(collumn, self.p1Symbol)
        elif player == 2:
            self.addToCollumn(collumn, self.p2Symbol)
            
    def addToCollumn(self, collumn, symbol):
        self.board[self.rowTops[collumn]][collumn] = symbol
        if self.checkWinner(symbol, (self.rowTops[collumn],collumn)):
            self.showWinner(symbol)
        self.rowTops[collumn] -= 1 #Vai decrementando os valores da lista rowTops
        
    def checkWinner(self, player, last_move):
        row, col = last_move
        token = player  # assuming player's token is represented by 'X' or 'O'

        # Check horizontal
        count = 0
        for c in range(self.cols):
            if self.board[row][c] == token:
                count += 1
                if count == 4:
                    return True
            else:
                count = 0

        # Check vertical
        count = 0
        for r in range(self.rows):
            if self.board[r][col] == token:
                count += 1
                if count == 4:
                    return True
            else:
                count = 0

        # Check diagonal up-left-to-right
        count = 0
        for i in range(-3, 4):
            if 0 <= row + i < self.rows and 0 <= col + i < self.cols:
                if self.board[row + i][col + i] == token:
                    count += 1
                    if count == 4:
                        return True
                else:
                    count = 0
        
        # Check diagonal down-left-to-right
        count = 0
        for i in range(-3, 4):
            if 0 <= row - i < self.rows and 0 <= col + i < self.cols:
                if self.board[row - i][col + i] == token:
                    count += 1
                    if count == 4:
                        return True
                else:
                    count = 0

                    
                    
    def showWinner(self, player):
        print("\n\nPLAYER "+str(player)+" WINS!\n"+str(self))
        self.end = True

**Game interface**

The interface for Connect Four uses the Pygame library for python to represent the game logic described above. It's based on a grid layout where players can drop their pieces. It uses two colors, red and blue, to identify the players and it respondes to mouse interaction. 

Developed with two classes, Piece and Board_Inetrface.

Class Piece - class used to create the game pieces. Initially, the board is filled with white pieces. As the players start playing, it's being draw a colored piece (red or blue) in the chosen place. 

In [19]:
class Piece:
    def __init__(self):
        self.square_size = 100
        self.radius = int(self.square_size / 2 - 5)
    
    def draw(self, screen, x, y, symbol):
        if symbol == 'X': 
            pygame.draw.circle(screen, (255, 0, 0), (x, y), self.radius)
        elif symbol == 'O':
            pygame.draw.circle(screen,(0, 0, 139), (x, y), self.radius)
        elif symbol == '.':
            pygame.draw.circle(screen,(255, 255, 255), (x, y), self.radius)

Class Board_Interface - Description

In [20]:

class Board_Interface:
    def __init__(self, row_count, column_count, square_size, board):
        self.row_count = row_count
        self.column_count = column_count
        self.square_size = square_size
        self.radius = self.radius = int(square_size / 2 - 5)
        self.width = 1200
        self.height = 700
        #interface colors
        self.colors = {
            "BLUE": (0, 0, 255),
            "BLACK": (0, 0, 0),
            "RED": (255, 0, 0),
            "YELLOW": (255, 255, 0),
            "WHITE": (255, 255, 255),
            "BACKGROUND": (230, 191, 131),
            "GRILL": (193, 154, 107),
            "EMPTY": (230, 230, 230)
        }
        #board
        self.game_board = board
        #screen
        self.screen = pygame.display.set_mode((self.width, self.height))
        #window name
        pygame.display.set_caption("CONNECT FOUR")
        #Clock
        self.clock = pygame.time.Clock()
        
        self.board_width = self.column_count * self.square_size #VERIFICADO
        self.board_height = self.row_count * self.square_size

        # Calculate the starting position to center the board
        self.start_x = (self.width - self.board_width) // 2
        self.start_y = (self.height - self.board_height + 50) // 2 #VERIFICADO

    def draw_board(self):
        #draw circles 
        for row in range(5,-1,-1):
            for column in range(self.game_board.cols):
                circle = Piece()
                circle_x = self.start_x + (column * self.square_size) + (self.square_size // 2)
                circle_y = self.start_y + (row * self.square_size) + (self.square_size // 2)
                symbol = self.game_board.board[row][column]
                circle.draw(self.screen, circle_x, circle_y, symbol)
                
        #draw grid
        for i in range(self.game_board.rows + 1):
            # Horizontal lines
            pygame.draw.line(self.screen, self.colors['GRILL'], (self.start_x, self.start_y + i * self.square_size),
                             (self.start_x + self.board_width, self.start_y + i * self.square_size), 3)
            
        for j in range(self.game_board.cols + 1):
            # Vertical lines
            pygame.draw.line(self.screen, self.colors['GRILL'], (self.start_x + j * self.square_size, self.start_y),
                             (self.start_x + j * self.square_size, self.start_y + self.board_height), 3)
       
                
        pygame.display.update()
            
    def convert_mouse_to_board_pos(self, x, y):
    # Ajustar para o início da grade do tabuleiro
        x -= self.start_x
        y -= self.start_y

        # Encontrar a coluna
        column = x // self.square_size

        # Encontrar a linha
        row = y // self.square_size

        return row, column
    
    def draw_text(self, text, font_size, color, x, y, duration):
        font = pygame.font.SysFont(None, font_size)
        text_surface = font.render(text, True, color)
        text_rect = text_surface.get_rect()
        text_rect.center = (x, y)
        self.screen.blit(text_surface, text_rect)
        
        pygame.display.update()
        pygame.time.delay(duration)


    def run_game(self):
        click = pygame.mouse.get_pressed()
        running = True
        game_turn = 0
        while running:
            self.draw_board()
            for event in pygame.event.get():
                # close game manually
                if event.type == pygame.QUIT:
                    running = False
                # close game using ESC
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        running = False

                # mouse touches
                
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    if event.button == 1:  # Left mouse button
                        x, y = pygame.mouse.get_pos()
                        row, column = self.convert_mouse_to_board_pos(x, y)
                        print(f'{row}, {column}')
                        # Handle the click here, e.g., updating the board
                        if game_turn == 0: 
                            self.game_board.addToCollumn(column, 'X')
                            game_turn = 1
                            print("entrou")
                            if self.game_board.checkWinner('X', (row, column)):
                                print("RED")
                                self.draw_text('Red Wins!', 70, (255, 0, 0), 600, 50, 5000)
                                self.clock.tick(60)
                                
                                running = False
                              
                        else: 
                            self.game_board.addToCollumn(column, 'O')
                            game_turn = 0
                         
                            if self.game_board.checkWinner('O', (row, column)):
                                self.draw_text('Blue Wins!', 70, (0, 0, 139), 600, 50, 5000)
                                self.clock.tick(60)
                                
                                running = False
                               
                        
                        print(self.game_board)
                        self.draw_board()
            
                    
            self.screen.fill(self.colors['BACKGROUND'])
        pygame.quit()




**Menu**

The menu uses the Pygame library for python to create a simple game menu with buttons for 2 players, player vs algorithm and algorithm vs algorithm. 

Developed with two classes, Button and Menu.


Class Button - creates the necessary buttons for the menu by drawing them on the screen and allowing them to respond to mouse interaction.

In [21]:
class Button:
    def __init__(self, text, width, height, font, text_color, backgroud_color, x_pos, y_pos):
        self.text = text
        self.width = width
        self.height = height
        self.font = font
        self.text_color = text_color
        self.background_color = backgroud_color
        self.x_pos = x_pos
        self.y_pos = y_pos
        
        self.colors = {'black': (0, 0, 0),
                       'white': ( 255, 255, 255),
                       'yellow': (255, 255, 0),
                       'red': (255, 0, 0),
                       'camel': (230, 191, 131),
                       'brown': (193, 154, 107)}
        self.rect = pygame.Rect(x_pos, y_pos, width, height)


    def draw(self, screen):
        pygame.draw.rect(screen, self.background_color, (self.x_pos, self.y_pos, self.width, self.height))
        text_surface = self.font.render(self.text, True, self.colors['black'])
        text_rect = text_surface.get_rect(center=(self.x_pos + self.width / 2, self.y_pos + self.height / 2))

        screen.blit(text_surface, text_rect)


    def collide_point(self, pos):
        return self.rect.collidepoint(pos)

Class Menu - description

In [22]:

class Menu:
    def __init__(self):
        self.width = 1024
        self.height = 700
        self.screen = pygame.display.set_mode((self.width, self.height))
        self.font = pygame.font.SysFont('Cambria', 35)
        self.button_font = pygame.font.SysFont('Arial', 15)
        self.menu_open = True
        self.colors = {'black': (0, 0, 0),
                       'white': ( 255, 255, 255),
                       'yellow': (255, 255, 0),
                       'red': (255, 0, 0),
                       'camel': (230, 191, 131),
                       'brown': (193, 154, 107)}
      
        self.player_vs_player_button = Button("PLAYER vs PLAYER", 175, 50, self.button_font, self.colors['black'], self.colors['brown'], self.width // 2 - 100, self.height // 2 - 25)
        self.astar_button = Button("A*", 175, 50, self.button_font, self.colors['black'], self.colors['brown'], self.width // 2 - 100, self.height // 2 + 50)
        self.monte_carlo_button = Button("Monte Carlo", 175, 50, self.button_font, self.colors['black'], self.colors['brown'], self.width // 2 - 100, self.height // 2 + 125)
        self.game_board = Board()
    def setup(self):
        self.screen.fill(self.colors['camel'])
        pygame.display.set_caption("Connect Four Game")

    def text(self, message, text_color, x_pos, y_pos):
        text = self.font.render(message, True, self.colors['black'])
        text_rect = text.get_rect(center=(x_pos, y_pos))
        self.screen.blit(text, text_rect)
    

    def run(self):
            self.setup()
            while self.menu_open:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        self.menu_open = False
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        if self.player_vs_player_button.collide_point(pygame.mouse.get_pos()):
                            self.start_game_interface()
                            
                            
                
                self.screen.fill(self.colors['camel']) 
                self.text("C O N N E C T    F O U R", self.colors['brown'], self.width // 2, self.height // 2 - 200)  
                self.player_vs_player_button.draw(self.screen)
                self.astar_button.draw(self.screen)
                self.monte_carlo_button.draw(self.screen)
                pygame.display.flip()

            pygame.quit()



    def start_game_interface(self):
        game_interface = Board_Interface(self.game_board.rows, self.game_board.cols, 100, self.game_board)
        game_interface.run_game()




Menu initialization

In [23]:
pygame.init()
menu = Menu()
menu.run()

# ALGORITHMS


**A-STAR ALGORITHM**

The first algorithm that we implemented was A*. This algorithm is an informed but non-adversarial strategy, therefore it does not take into account the fact
that an adversary will change the state of the system in the next steps. The objective of this algorithm is to find the shortest path from a starting point to a goal point in a graph, considering the edge weights and a heuristic function. 

**COMPONENTS**

**F(n) = G(n) + H(n)**

- F(n) = The total estimated cost of the cheapest path from the start node to the goal node that passes through node n
- G(n) = The cost of the path from starting node to node n
- H(n) = Estimated cost of the cheapest path from node n to the goal (heuristic function)
- The tree that is needed to generate nodes 

**A*STAR TREE AND NODE**

The object Node receives: 
- a value (a matrix of the board in AStarTree)
- a dictionary that represents the children of that same node {0: node0, 1: node1,...}
- a parent node which is a node too and represents the parent node of the current node

We have setters and getters to make the operations easier. Also we have a genChildren function that 

In [15]:
class Node:
    def __init__(self, value, children, parent):
        self.value = value      # is a matrix of the board in AStarTree and is a Board object in MCTree
        self.children = children    # is a dictionary {0: node, 1: node, ...}
        self.parent = parent    # is another node
        self.wins = 0
        self.visits = 0
        
    def setValue(self, value):
        self.value = value
        
    def setChildren(self, children):
        self.children = children
        
    def getChild(self, child):
        return self.children[child]
        
    def setParent(self, parent):
        self.parent = parent
    
    def getParent(self):
        return self.parent
    
    #def win(self):
        self.wins += 1
        
    #def visit(self):
        self.visits += 1
        
    def genChildren(self, player):
        children = {}
        for i in self.value.possibleMoves():
            temporaryBoard = board.Board()
            temporaryBoard.board = np.copy(self.value.board)
            temporaryBoard.winner = self.value.winner
            temporaryBoard.end = self.value.end
            temporaryBoard.move(i, player)
            
            children[i] = Node(temporaryBoard, {}, self)
        self.setChildren(children)
    
    def nodeHeight(self):
        return self.calculateNodeHeight(self)

    def calculateNodeHeight(self, node):
        if node.parent == None: return 0
        else:
            return self.calculateNodeHeight(node.parent) + 1
        
    def copy(self):
        boardCopy = board.Board()
        boardCopy.board = np.copy(self.value.board)
        boardCopy.winner = self.value.winner
        boardCopy.end = self.value.end

        childrenCopy = {}
        for child in self.children:
            childrenCopy[child] = self.getChild(child).copy()
        
        node = Node(boardCopy, childrenCopy, self.parent)
        node.wins = self.wins
        node.visits = self.visits
        
        return node

In [16]:
class AStarTree:
    def __init__(self, board, player):              # player is 1 or 2, board is a board object
        self.player = player                        # 1 or 2
        self.root = Node(board, {}, None)           # root node with the matrix of the board as its value
        self.root.genChildren(player)               # generates new children for the root

We chose to develop two heuristics:
- the one that was provided by the teacher 
- other that focuses on the evaluation of consecutive pieces 